# IMagic

<p>
IMagic is a Deep Learning based model to color a black and white image i.e. by converting a Grayscale image into RGB image
</p>
### Python libraries used
<br>
<dl>
    <dt>Tensorflow</dt>
    <dd><div style="margin-left:20px;">Tensorflow is an open-source software library for dataflow programming across a range of tasks. It is a symbolic math library, and is also used for machine learning applications such as Neural Networks.</div></dd>
    <dt>Numpy</dt>
    <dd><div style="margin-left:20px;">NumPy is the fundamental package for scientific computing with Python. NumPy can also be used as an efficient multi-dimensional container of generic data. </div></dd>
    <dt>Open-cv</dt>
    <dd><div style="margin-left:20px;">OpenCV (Open Source Computer Vision) is a library of programming functions mainly aimed at real-time computer vision. Used as an image processing library in this program.</div></dd>
    <dt>Matplotlib</dt>
    <dd><div style="margin-left:20px;">Matplotlib is a plotting library for the Python programming language and its numerical mathematics extension NumPy. The matplotlib.pyplot module contains functions that allow you to generate many kinds of plots quickly.</div></dd>
    <dt>h5py</dt>
    <dd><div style="margin-left:20px;">The h5py package is a Pythonic interface to the HDF5 binary data format.

It lets you store huge amounts of numerical data, and easily manipulate that data from NumPy. For example, you can slice into multi-terabyte datasets stored on disk, as if they were real NumPy arrays. Thousands of datasets can be stored in a single file, categorized and tagged however you want.</div></dd>
</dl>

### Dataset Description

The datasets are image files that can be read using the open-cv python library. Each sample image used was originally 256x256 pixels and consists of 3 bands - red, green, and blue. These images are resized into 64x64 pixels images and converted seperately into Grayscale image which could be used as input and orginal 3 bands image as an output to neural network. 



### Methodology

This model follows the methodology of Supervised Learning using Regression where we have to predict a particular value for input features.</b><br>
Neural network used for converting grayscale image to RGB image takes a 64X64 pixels grayscale image as input and 64x64x3 pixels RGB image as an output. These images are converted into single vertor before feeding into the neural network. Then model is trained over these images repeatedly to achieve the minimal loss and highest accuracy.

<div style="height:100px;width:200px;"></div>


#### Importing Tensorflow and other libraries

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import h5py

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Loading,Resizing  and flattening the Images

In [2]:
mn=721
Y=np.zeros((36237,12288))
X=np.zeros((36237,4096))
k=0

dir='datac/dataa/train/'
for filename in os.listdir(dir):
    #file='res/image1/image{0}.jpg'.format(var+1)
    img=cv2.imread(dir+filename)
    img=cv2.resize(img,(64,64))
    im=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).reshape(1,-1)
    X[k]=im.reshape(1,-1)
    Y[k]=img.reshape(1,64*64*3)
    k=k+1
    if k==721:
        break
    

#### Dividing the dataset into Training and Test set


In [3]:
nm=721
m=int(np.floor(90*nm/100))

X_train=X[0:m,:].T
Y_train=Y[0:m,:].T
X_test=X[m:nm,:].T
Y_test=Y[m:nm,:].T
print('X_train if of size {}'.format(X_train.shape))
print('Y_train if of size {}'.format(Y_train.shape))
print('X_test if of size {}'.format(X_test.shape))
print('Y_test if of size {}'.format(Y_test.shape))
print('Total examples : {}'.format(Y_test.shape[1]+Y_train.shape[1]))


X_train if of size (4096, 648)
Y_train if of size (12288, 648)
X_test if of size (4096, 73)
Y_test if of size (12288, 73)
Total examples : 721


#### Creating placeholder to pass training data

In [4]:
def create_placeholder(n_x,n_y):
    X=tf.placeholder(dtype=tf.float32,shape=[n_x,None],name='X')
    Y=tf.placeholder(dtype=tf.float32,shape=[n_y,None],name='Y')
    
    return X,Y


In [5]:
def fetch_parameters(param):
    
    W1=tf.Variable(param['W1'],[12288,4096],dtype=tf.float32)
    b1=tf.Variable(param['b1'],[12288,1],dtype=tf.float32)
    W2=tf.Variable(param['W2'],[12288,12288],dtype=tf.float32)
    b2=tf.Variable(param['b2'],[12288,1],dtype=tf.float32)
    
    #print(type(param['W1']))
    #W1=param['W1'],
    #b1=param['b1'],
    #W2=param['W2'],
    #b2=param['b2']
    
    parameters={
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    
    return parameters


#### Intializing the parameters i.e. defining the weights and bias

In [6]:
def parameters():
    
    #Total no. of layers in network : 3
    #1 input layer, 1 hidden layer, 1 output layer
    
    #Size of layer 1 : 4096
    #Size of layer 2: 12288
    #Size of layer 3: 12288
    
    #W1(Weights of input layer) : [12288,4096]
    #b1(bias of input layer) : [12288,1]
    #W2(Weights of hidden layer) : [12288,12288]
    #b2(bias of hidden layer) : [12288,1]
    
    
    W1=tf.get_variable("W1",[12288,4096],initializer=tf.contrib.layers.xavier_initializer(seed=1),dtype=tf.float32)
    b1=tf.get_variable("b1",[12288,1],initializer=tf.contrib.layers.xavier_initializer(seed=1),dtype=tf.float32)
    W2=tf.get_variable("W2",[12288,12288],initializer=tf.contrib.layers.xavier_initializer(seed=1),dtype=tf.float32)
    b2=tf.get_variable("b2",[12288,1],initializer=tf.contrib.layers.xavier_initializer(seed=1),dtype=tf.float32)
    
    parameters={
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    
    return parameters


#### Forward propagation

In [7]:
def forward(X,parameters):
    
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']
    b2=parameters['b2']
   
    A1=tf.add(tf.matmul(W1,X),b1)
    A2=tf.add(tf.matmul(W2,A1),b2)
    
    return A2


#### Computing the cost

In [8]:
def get_cost(A2,Y):
    x=tf.transpose(A2)
    y=tf.transpose(Y)
    
    cost=tf.reduce_mean(tf.square(x-y))
    
    return cost


#### Creating batch for training the model

In [9]:
def batching(X,Y,batch_size=64,seed=0):
    
    m=X.shape[1]
    mini_batches=[]
    np.random.seed(seed)
    
    #Permutate the training data
    per=np.random.permutation(m)
    x_shuffled=X[:,per]
    y_shuffled=Y[:,per]
    
    #partitioning
    batches=int(np.floor(m/batch_size))
    
    for var in range(batches):
        batch_x=x_shuffled[:,var*batch_size:var*batch_size+batch_size]
        batch_y=y_shuffled[:,var*batch_size:var*batch_size+batch_size]
        mini_batches.append((batch_x,batch_y))
        
    if m%batch_size!=0:
        batch_x=x_shuffled[:,batches*batch_size:m]
        batch_y=y_shuffled[:,batches*batch_size:m]
        mini_batches.append((batch_x,batch_y))
        #print(len(mini_batches))
    
    return mini_batches


#### Main function for training the model

In [25]:
def model(X_train,Y_train,X_test,Y_test,learning_rate=0.000001,epochs=50,mini_batch_size=32,param=None):
    
    jl=0
    tf.reset_default_graph()
    tf.set_random_seed(1)
    seed=3
    (n_x,m)=X_train.shape
    (n_y,m)=Y_train.shape
    
    #create palceholders 
    X,Y=create_placeholder(n_x,n_y)
    costs=[]

    #para=parameters()
    para=fetch_parameters(param)
    
    A2=forward(X,para)
    
    cost=get_cost(A2,Y)
    
    #Backpropagation
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    #Initialize all variables
    init=tf.global_variables_initializer()
    
    #Start session
    with tf.Session() as session:
        
        #Run session
        session.run(init)
        
        for epoch in range(epochs):
            
            var=epoch
            
            if var!=0:
                os.remove('track_{}.txt'.format(str(var)))
           
            fp=open('track_{}.txt'.format(str(var+1)),'w')
            fp.write('Iteration no. {}\r\n'.format(var+1))
            fp.close()
            
            num_minibatches = int(m / mini_batch_size)
            epoch_cost=0
            seed=seed+1
            minibatches=batching(X_train,Y_train,mini_batch_size,seed)
            
            kl=1
            for minibatch in minibatches:
                
                #Select a batch
                (mini_X,mini_y)=minibatch
                
                #Main Line to run the code
                _, mini_cost=session.run([optimizer, cost], feed_dict={X:mini_X,Y:mini_y})
                
                epoch_cost+=mini_cost/num_minibatches
                
                if jl!=0:
                    os.remove('track_b{}.txt'.format(str(jl)))
           
                fp=open('track_b{}.txt'.format(str(kl)),'w')
                fp.write('Batch no. {0}\r\nEpoch cost {1}'.format(kl,epoch_cost))
                fp.close()
                jl=kl
                #kl=kl+1
                
            if epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if epoch % 5 == 0:
                costs.append(epoch_cost)
                univCost.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        
        # lets save the parameters in a variable
        parameter = session.run(para)
        print ("Parameters have been trained!")
                
        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(A2), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameter,costs        
            

#### Read the pre-trained parameters

In [21]:
param={}
filename = 'params1.hdf5'
f = h5py.File(filename, 'r')
param['W1']=f['w1'][()]
param['b1']=f['b1'][()]
param['W2']=f['w2'][()]
param['b2']=f['b2'][()]
univCost=list(f['cost'][()])
f.close()

#### Training the model

In [ ]:
param,costs = model(X_train, Y_train, X_train, Y_train,param=param)

Cost after epoch 0: 10.427301
Cost after epoch 1: 9.253550
Cost after epoch 2: 8.915696
Cost after epoch 3: 9.778020
Cost after epoch 4: 9.174371
Cost after epoch 5: 9.297854
Cost after epoch 6: 9.224076
Cost after epoch 7: 9.256226
Cost after epoch 8: 8.966103
Cost after epoch 9: 11.712112
Cost after epoch 10: 9.298902
Cost after epoch 11: 9.022528
Cost after epoch 12: 8.968265
Cost after epoch 13: 8.881223
Cost after epoch 14: 8.931230
Cost after epoch 15: 8.773939
Cost after epoch 16: 8.996440
Cost after epoch 17: 9.053765
Cost after epoch 18: 8.912380
Cost after epoch 19: 8.914247
Cost after epoch 20: 9.179793
Cost after epoch 21: 9.049880
Cost after epoch 22: 9.416660
Cost after epoch 23: 9.338563
Cost after epoch 24: 9.307692
Cost after epoch 25: 9.845260
Cost after epoch 26: 9.026983
Cost after epoch 27: 8.720493
Cost after epoch 28: 8.801162
Cost after epoch 29: 9.086967


#### Saving the trained parameters

In [24]:
univCost.append(404)
f=h5py.File('params1.hdf5','w')
f.create_dataset('w1',data=np.array(param['W1']))
f.create_dataset('w2',data=np.array(param['W2']))
f.create_dataset('b1',data=np.array(param['b1']))
f.create_dataset('b2',data=np.array(param['b2']))
f.create_dataset('cost',data=np.array(univCost))
f.close()